In [182]:
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import os
import pandas as pd
import numpy as np

In [183]:
#suppress tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [184]:
#The df
untouched_csv = pd.read_csv('CSV/RAW_data.csv', sep=',')

In [185]:
#replace the direction column with the corresponding degree values
direction_map = {'N': 0,'NNE': 22.5,'NE': 45,'ENE': 67.5,'E': 90,'ESE': 112.5,'SE': 135,'SSE': 157.5,
            'S': 180,'SSW': 202.5,'SW': 225,'WSW': 247.5,'W': 270,'WNW': 292.5,'NW': 315,'NNW': 337.5
        }

In [186]:
untouched_csv['Direction (A)'] = untouched_csv['Direction (A)'].replace(direction_map).infer_objects(copy = False)
#untouched_csv['Direction (A)'] = untouched_csv['Direction (A)'].astype(int)

C:\Users\Krupam\AppData\Local\Temp\ipykernel_25504\3364744739.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  untouched_csv['Direction (A)'] = untouched_csv['Direction (A)'].replace(direction_map).infer_objects(copy = False)


In [187]:
#convert to Celsius
untouched_csv['Mainland Air Temp'] = round((untouched_csv['Mainland Air Temp']-32) * 5.0/9.0, 1)
untouched_csv['LBI Air Temp'] = round((untouched_csv['LBI Air Temp']-32) * 5.0/9.0, 1)


In [188]:
#rounds all colomns
untouched_csv['Humidity (%)'] = round(untouched_csv['Humidity (%)'], 1)
untouched_csv['Wind Speed (A)'] = round(untouched_csv['Wind Speed (A)'], 1)
untouched_csv['Gusting'] = round(untouched_csv['Gusting'], 1)
untouched_csv['Atmospheric Pressure (IN)'] = round(untouched_csv['Atmospheric Pressure (IN)'], 2)
untouched_csv['Precipitation Rate'] = round(untouched_csv['Precipitation Rate'], 2)
untouched_csv['Bay Temp'] = round(untouched_csv['Bay Temp'], 2)
untouched_csv['Salinity'] = round(untouched_csv['Salinity'], 2)
untouched_csv['LBI Air Temp'] = round(untouched_csv['LBI Air Temp'], 1)
untouched_csv['Ocean Temp'] = round(untouched_csv['Ocean Temp'], 1)

In [189]:
#determines if its a onshore breeze and adds a new column for it
onshore_degrees = [180, 135, 157.5, 90, 67.5, 45, 22.5]
untouched_csv['Onshore'] = untouched_csv['Direction (A)'].isin(onshore_degrees)
untouched_csv['Onshore'] = untouched_csv['Onshore'].astype(int)



In [190]:
#thresholds
ocean_thresh = 1.0
bay_thresh = 1.67
wind_thresh = 7.0
untouched_csv['rolling avg'] = untouched_csv['Bay Temp'].rolling(window=24).mean()

#Boolean math to check if it is upwelling or not
untouched_csv['bay_bool'] = (abs(untouched_csv['Bay Temp'] - untouched_csv['rolling avg']) > bay_thresh).astype(int)
untouched_csv['ocean_bool'] = (abs(untouched_csv["Bay Temp"] - untouched_csv["Ocean Temp"]) > ocean_thresh).astype(int)
untouched_csv['big_wind'] = (untouched_csv['Wind Speed (A)'] > wind_thresh).astype(int)
untouched_csv["upwelling_flag"] = ((untouched_csv['bay_bool'] == 1) & (untouched_csv['ocean_bool'] == 1) & (untouched_csv['big_wind'] == 1)).astype(int)
"""Explain this here"""

#Drop the temporary colomns
untouched_csv.drop(['bay_bool', 'ocean_bool', 'big_wind', 'rolling avg'], axis=1, inplace=True)
	
	
    

In [191]:
#Convert data into Date time and create date filter
untouched_csv['Date'] = pd.to_datetime(untouched_csv['Date'])

"""
Use this in future if data set needs specific dates
prediction = data.loc{
    (untouched_csv['Date'] > datetime(x, x, x)) &
    (untouched_csv['Date'] < datetime(x, x, x,))
}
"""

"\nUse this in future if data set needs specific dates\nprediction = data.loc{\n    (untouched_csv['Date'] > datetime(x, x, x)) &\n    (untouched_csv['Date'] < datetime(x, x, x,))\n}\n"

In [192]:
#Save this data frame to a Finished csv

In [193]:
#untouched_csv.head()

#info on csv
#print(untouched_csv.info())

#Number of upwelling events
"""
num_upwelling_events = untouched_csv['upwelling_flag'].sum()
print("Number of upwelling hours:", num_upwelling_events)
"""

#Find which rows where upwelling
"""
upwelling_rows = untouched_csv[untouched_csv['upwelling_flag'] == 1]
print(upwelling_rows)
"""

#print(untouched_csv.to_string())

"\nupwelling_rows = untouched_csv[untouched_csv['upwelling_flag'] == 1]\nprint(upwelling_rows)\n"

In [194]:
#Prepare for the LSTM model (Sequential)
